In [1]:
##########################################
# Modifications inside the sponge zone (outflow) to meet bc requirement #
# Zhenyang Yuan                          #
# 25/5/2021                              #
##########################################
import numpy as np
from math import pi
import math
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
import scipy.signal
from numpy import linalg as LA
#from matplotlib.mlab import griddata
params = {'legend.fontsize': 15,
          'legend.loc':'best',
          'figure.figsize': (14,5),
          'lines.markerfacecolor':'none',
         'axes.labelsize': 17,
         'axes.titlesize': 17,
         'xtick.labelsize':15,
         'ytick.labelsize':15,
         'grid.alpha':0.6}
pylab.rcParams.update(params)
#%matplotlib notebook
%matplotlib inline


from sys import argv

import pandas as pd

In [2]:
# load data set
def data_load(headline,varibles,numNodes,Lines):
    

    x = np.asarray([])
    y = np.asarray([])
    rho = np.asarray([])
    rhou = np.asarray([])
    rhov = np.asarray([])
    E = np.asarray([])
    
    
        
    

    k = 0
    #for i in open("../send/"+str(time)+".dat").readlines():
    for i in open("test.dat").readlines():
        if k > headline - 1:                                #get away with headers
            datContent = i.strip().split()                  #magic

            datContent = np.asarray(datContent)             #convert list to array
            datContent = datContent.astype(np.float64)      #convert str element to float

           
            if k < Lines*1 + headline :
                    x = np.concatenate((x,datContent),axis=None)
            elif k < Lines*2 + headline :
                    y = np.concatenate((y,datContent),axis=None)
            elif k < Lines*3 + headline :
                    rho = np.concatenate((rho,datContent),axis=None)
            elif k < Lines*4 + headline :
                    rhou = np.concatenate((rhou,datContent),axis=None)
            elif k < Lines*5 + headline :
                    rhov = np.concatenate((rhov,datContent),axis=None)
            elif k < Lines*6 + headline :
                    E = np.concatenate((E,datContent),axis=None)
            else:
                break
        k += 1

 

    print('Completed Reading data!\n')

    return x,y,rho,rhou,rhov,E
    


In [3]:
def data_process(x_,y_,rho_,rhou_,rhov_,E_):
    
    
    x = np.asarray([])
    y = np.asarray([])
    rho = np.asarray([])
    rhou = np.asarray([])
    rhov = np.asarray([])
    E = np.asarray([])
    
    index = np.asarray([])
    
            
    
    for i in range(4,len(x_)-4):
        if x_[i] != x_[i+1] and y_[i] != y_[i+1] and x_[i] != x_[i-1] and y_[i] != y_[i-1]:
            index = np.concatenate((index,i),axis=None)
        elif x_[i] == x_[i+1] and y_[i] == y_[i+1] and x_[i] != x_[i-1] and y_[i] != y_[i-1]:
            index = np.concatenate((index,i),axis=None)
            
    index = index.astype(np.int32)
    
    x = x_[index]
    y = y_[index]
    rho = rho_[index]
    rhou = rhou_[index]
    rhov = rhov_[index]
    E = E_[index]
    
                
    return x,y,rho,rhou,rhov,E
    

In [4]:
headline=20
varibles=6   # x-1, y-2, rho-3, rhou-4, rhov-5, E-6, u-7, v-8, p-9, T, s, a, Mach, Sensor
numNodes=95295
Lines=int(numNodes/5) #from tecplot
#Lines=int(numNodes/1000)+1 #fieldconvert



x_,y_,rho_,rhou_,rhov_,E_ = data_load(headline,varibles,numNodes,Lines)

x,y,rho,rhou,rhov,E = data_process(x_,y_,rho_,rhou_,rhov_,E_)





Completed Reading data!



In [5]:
def Modyfication(x,var,var_ref,xmin,xmax,method):
    # step function:
    f = lambda x_: 6*((x_-xmin)/(xmax-xmin))**5 - 15*((x_-xmin)/(xmax-xmin))**4 +10*((x_-xmin)/(xmax-xmin))**3
    
    # tanh function
    #f2 = lambda x:
    
    #sponge zone modification
    for i in range(len(x)):
        if x[i] >= xmin and x[i] <= xmax:
            var[i] = var[i] - (var[i] - var_ref)*f(x[i])
    
    return var

In [6]:
rhoInf = 1.0
Mach = 0.3
cInf = 1.0
Gamma = 1.4
pInf = cInf*cInf*rhoInf/Gamma
vInf = 0.0
uInf = Mach*cInf

rho_ref = rhoInf
rhou_ref = rhoInf*uInf
rhov_ref = rhoInf*0.0
E_ref = pInf/(Gamma-1)+0.5*rhoInf*(uInf*uInf+vInf*vInf)

xmin = 8
xmax = 10

method = 1

In [7]:
rho = Modyfication(x,rho,rho_ref,xmin,xmax,method)
rhou = Modyfication(x,rhou,rhou_ref,xmin,xmax,method)
rhov = Modyfication(x,rhov,rhov_ref,xmin,xmax,method)
E = Modyfication(x,E,E_ref,xmin,xmax,method)

In [8]:
def write2pts(x,y,rho,rhou,rhov,E):
    f = open("ModyfiedField.pts", "a")

    f.write('<?xml version="1.0" encoding="utf-8"?>\n')
    f.write('<NEKTAR>\n')
    f.write('    <POINTS DIM="2" FIELDS="rho,rhou,rhov,E">\n')
    for i in range(len(x)):
        f.write('        '+str(x[i])+'    '+str(y[i])+'    '+str(rho[i])+'    '+str(rhou[i])+'    '+str(rhov[i])+'    '+str(E[i])+'\n')
    f.write('    </POINTS>\n')
    f.write('</NEKTAR>')   
    f.close()

In [9]:
write2pts(x,y,rho,rhou,rhov,E)

In [31]:
# others misc
layer = 51
T = 0.5
ds = 0.0004   # yplus = 1.0
p = 1.1
temp = 0
for i in range(layer):
    temp += p**i
    
ds = T/temp
print(ds)

0.0003902288623176589
